Import Libraries and increase max rows and columns to see all data on output

In [ ]:
#import libraries
import pandas as pd
import xlwings as xw
import os
import openpyxl as op
import numpy as np
import seaborn as sns
from tkinter import Tk
from tkinter import filedialog
from tkinter import * 
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

Select folder where data is located and create list of file names

In [ ]:
root = Tk()
root.withdraw()
#select folder path to use
folderSelected = filedialog.askdirectory()
files = os.listdir(folderSelected)

In [ ]:
def get_sec(time_str): #function for formatting HH:MM:SS string to just seconds https://stackoverflow.com/questions/6402812/how-to-convert-an-hmmss-time-string-to-seconds-in-python
    """Get seconds from time."""
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

Create new columns for High intensity running

In [ ]:
dfCollection = []
x = 0 #this instead of enumerate incase more files than desired
for file in files:
    if file.rfind(".csv") > 0:
        currentFile = folderSelected + "/" + file
        dfCollection.append(pd.read_csv(currentFile))
        nameStartLoc = currentFile.rfind("/") +1
        dfCollection[x].name = currentFile[nameStartLoc:].replace(".csv", "")
        
        dfCollection[x].rename(columns = {"Distance in Speed zone 3 [m] (15.00 - 19.79 km/h)":"High Intensity Running (15.00 - 19.79 km/h) (m)", "Distance in Speed zone 4 [m] (19.80 - 25.19 km/h)":"High Speed Running (19.80 - 25.19 km/h) (m)", "Distance in Speed zone 5 [m] (25.20- km/h)":"Sprinting (25.20+ km/h) (m)"}, inplace = True)
        #sum up zone 3,4,5 running
        dfCollection[x]["Total High Intensity Sum (>15km/h) (m)"] = dfCollection[x]["High Intensity Running (15.00 - 19.79 km/h) (m)"] + dfCollection[x]["High Speed Running (19.80 - 25.19 km/h) (m)"] + dfCollection[x]["Sprinting (25.20+ km/h) (m)"]

        #create relative percentages for different speeds of running
        dfCollection[x]["Relative High Intensity Running (% of Total Running)"] = (dfCollection[x]["High Intensity Running (15.00 - 19.79 km/h) (m)"] / dfCollection[x]["Total distance [m]"]) *100
        dfCollection[x]["Relative High Speed Running (% of Total Running)"] = (dfCollection[x]["High Speed Running (19.80 - 25.19 km/h) (m)"]/ dfCollection[x]["Total distance [m]"]) * 100
        dfCollection[x]["Relative Sprinting (% of Total Running)"] = (dfCollection[x]["Sprinting (25.20+ km/h) (m)"]/ dfCollection[x]["Total distance [m]"]) *100
        dfCollection[x]["Relative High Intensity Sum (% of Total Running)"] = (dfCollection[x]["Total High Intensity Sum (>15km/h) (m)"]/ dfCollection[x]["Total distance [m]"]) *100
        
        #change type object for times in power zones
        dfCollection[x]["Time in power zone 1 (70 - 84 %)"] = dfCollection[x]["Time in power zone 1 (70 - 84 %)"].astype("string")
        dfCollection[x]["Time in power zone 1 (70 - 84 %)"] = dfCollection[x]["Time in power zone 1 (70 - 84 %)"].apply(get_sec)
        
        dfCollection[x]["Time in power zone 2 (85 - 99 %)"] = dfCollection[x]["Time in power zone 2 (85 - 99 %)"].astype("string")
        dfCollection[x]["Time in power zone 2 (85 - 99 %)"] = dfCollection[x]["Time in power zone 2 (85 - 99 %)"].apply(get_sec)
        
        dfCollection[x]["Time in power zone 3 (100 - 129 %)"] = dfCollection[x]["Time in power zone 3 (100 - 129 %)"].astype("string")
        dfCollection[x]["Time in power zone 3 (100 - 129 %)"] = dfCollection[x]["Time in power zone 3 (100 - 129 %)"].apply(get_sec)
        
        dfCollection[x]["Time in power zone 4 (130 - 179 %)"] = dfCollection[x]["Time in power zone 4 (130 - 179 %)"].astype("string")
        dfCollection[x]["Time in power zone 4 (130 - 179 %)"] = dfCollection[x]["Time in power zone 4 (130 - 179 %)"].apply(get_sec)
        
        dfCollection[x]["Time in power zone 5 (180 - 800 %)"] = dfCollection[x]["Time in power zone 5 (180 - 800 %)"].astype("string")
        dfCollection[x]["Time in power zone 5 (180 - 800 %)"] = dfCollection[x]["Time in power zone 5 (180 - 800 %)"].apply(get_sec)
    
        x=x+1

Combine first and second halfves into full game

In [ ]:
columnNamesList = []
firstHalf = []
secondHalf = []
fullGame = []
for col in dfCollection[1].columns:
    columnNamesList.append(col)
    fullGame.append([])
    firstHalf.append(dfCollection[1][col].values)
for col in dfCollection[2].columns:
    secondHalf.append(dfCollection[2][col].values)
#might be way to iterate through original list names and create logic off of strings to allow for differing data to be inputted
for iteration, x in enumerate(firstHalf):
    for iterationTwo, y in enumerate(x):
        #Names
        if iteration == 0:
            fullGame[iteration].append(firstHalf[iteration][iterationTwo])
        #HRMIN
        if iteration == 1:
            fullGame[iteration].append(min(firstHalf[iteration][iterationTwo], secondHalf[iteration][iterationTwo]))
        #HRAVG
        if iteration == 2:
            fullGame[iteration].append((firstHalf[iteration][iterationTwo] + secondHalf[iteration][iterationTwo])/2)
        #HRMAX and Max Speed
        if iteration == 3 or iteration == 5:
            localMax = max(firstHalf[iteration][iterationTwo], secondHalf[iteration][iterationTwo])
            if localMax > 100: #cleans data, anything over 100 is just 100
                print(localMax)
                localMax = 100
            fullGame[iteration].append(localMax)
        #Total distance and Sprints	Distance in Speed zone 1 [m] (3.00 - 11.99 km/h)	Distance in Speed zone 2 [m] (12.00 - 14.99 km/h)	High Intensity Running (15.00 - 19.79 km/h) (m)	High Speed Running (19.80 - 25.19 km/h) (m)	Sprinting (25.20+ km/h) (m)	Number of accelerations (-50.00 - -3.00 m/s²)	Number of accelerations (-2.99 - -2.00 m/s²)	Number of accelerations (-1.99 - -1.00 m/s²)	Number of accelerations (-0.99 - -0.50 m/s²)	Number of accelerations (0.50 - 0.99 m/s²)	Number of accelerations (1.00 - 1.99 m/s²)	Number of accelerations (2.00 - 2.99 m/s²)	Number of accelerations (3.00 - 50.00 m/s²)	Training load score	Cardio load	Time in power zone 1 (70 - 84 %)	Time in power zone 2 (85 - 99 %)	Time in power zone 3 (100 - 129 %)	Time in power zone 4 (130 - 179 %)	Time in power zone 5 (180 - 800 %)	Muscle load	Total High Intensity Sum (>15km/h) (m)
        if iteration == 4 or (iteration >= 6 and iteration <= 28):
            fullGame[iteration].append((firstHalf[iteration][iterationTwo] + secondHalf[iteration][iterationTwo]))
        #Relative High Intensity Running (% of Total Running)
        if iteration == 29:
            fullGame[iteration].append((fullGame[9][iterationTwo])/(fullGame[4][iterationTwo])*100)
        #Relative High Speed Running (% of Total Running)
        if iteration == 30:
            fullGame[iteration].append((fullGame[10][iterationTwo])/(fullGame[4][iterationTwo])*100)
        #Relative Sprinting (% of Total Running)
        if iteration == 31:
            fullGame[iteration].append((fullGame[11][iterationTwo])/(fullGame[4][iterationTwo])*100)
        #Relative High Intensity Sum (% of Total Running)
        if iteration == 32:
            fullGame[iteration].append((fullGame[28][iterationTwo])/(fullGame[4][iterationTwo])*100)

fullGame.append([])
columnNamesList.append("Game ID")

for x in fullGame[0]:
    fullGame[33].append(folderSelected[(folderSelected.rfind("/")+1):])

df = pd.DataFrame(fullGame)
df = df.transpose()
df.columns = columnNamesList
dfCollection.append(df)

Write to Excel by iterating through list of files and checking to see if they are a csv

In [ ]:
excelName = folderSelected + "/" + folderSelected[(folderSelected.rfind("/")+1):] + ".xlsx"

with pd.ExcelWriter(excelName) as writer:
    i = 0
    for x in files:
        if x.rfind(".csv") > 0:
            x = x[(x.rfind("/") + 1):].replace(".csv", "")
            dfCollection[i].to_excel(writer,sheet_name=x)
            i= i+1
        if i == 3:
             dfCollection[i].to_excel(writer,sheet_name="full-game")
             
wkb = xw.books.open(excelName)
i = 0
for x in files:
    if x.rfind(".csv") > 0:
        xw.view(dfCollection[i], sheet = wkb.sheets(x.replace(".csv", "")))
        i = i+1
    if i == 3:
        xw.view(dfCollection[i], sheet = wkb.sheets("full-game"))

Create full game CSV - to write to master sheet later

In [ ]:
#dfCollection[3].to_csv("full-game.csv", header= False)
csvName = folderSelected + "/full-game.csv"
dfCollection[3].to_csv(csvName)